In [1]:
# Step 1: Setup and imports
import sqlite3
import os
import pandas as pd

In [2]:
db_path = "crypto.db"

# Delete existing database file if it exists
if os.path.exists(db_path):
    os.remove(db_path)
    print("Database file deleted.")
else:
    print("Database file does not exist.")

# Create or connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

Database file deleted.


In [3]:
# Create tables

# Table for coin details
cursor.execute("""
CREATE TABLE IF NOT EXISTS coins (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    symbol TEXT NOT NULL,
    market_cap REAL,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

# Table for daily prices
cursor.execute("""
CREATE TABLE IF NOT EXISTS prices (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    coin_id INTEGER NOT NULL,
    date TEXT NOT NULL,
    price REAL NOT NULL,
    FOREIGN KEY (coin_id) REFERENCES coins(id)
)
""")

conn.commit()
print("Tables created successfully.")


Tables created successfully.


In [4]:
# Step 3: Insert at least 3 coins into `coins` table
coins = [
    ("Bitcoin", "BTC", 950000000000),
    ("Ethereum", "ETH", 350000000000),
    ("Cardano", "ADA", 50000000000)
]

cursor.executemany("""
INSERT INTO coins (name, symbol, market_cap) VALUES (?, ?, ?)
""", coins)
conn.commit()

# Print inserted coins
df_coins = pd.read_sql_query("SELECT * FROM coins", conn)
print("Coins inserted into 'coins' table:")
print(df_coins)


Coins inserted into 'coins' table:
   id      name symbol    market_cap         last_updated
0   1   Bitcoin    BTC  9.500000e+11  2025-08-13 17:14:45
1   2  Ethereum    ETH  3.500000e+11  2025-08-13 17:14:45
2   3   Cardano    ADA  5.000000e+10  2025-08-13 17:14:45


In [5]:
# Step 4: Insert 5 daily price records for each coin into `prices` table
prices_data = [
    # Bitcoin (id = 1)
    (1, "2025-08-01", 50000),
    (1, "2025-08-02", 50500),
    (1, "2025-08-03", 49000),
    (1, "2025-08-04", 51500),
    (1, "2025-08-05", 52000),

    # Ethereum (id = 2)
    (2, "2025-08-01", 3000),
    (2, "2025-08-02", 3050),
    (2, "2025-08-03", 3100),
    (2, "2025-08-04", 2950),
    (2, "2025-08-05", 3200),

    # Cardano (id = 3)
    (3, "2025-08-01", 1.5),
    (3, "2025-08-02", 1.55),
    (3, "2025-08-03", 1.6),
    (3, "2025-08-04", 1.45),
    (3, "2025-08-05", 1.65)
]

cursor.executemany("""
INSERT INTO prices (coin_id, date, price) VALUES (?, ?, ?)
""", prices_data)
conn.commit()

# Print inserted prices
df_prices = pd.read_sql_query("SELECT * FROM prices", conn)
print("Prices inserted into 'prices' table:")
print(df_prices)


Prices inserted into 'prices' table:
    id  coin_id        date     price
0    1        1  2025-08-01  50000.00
1    2        1  2025-08-02  50500.00
2    3        1  2025-08-03  49000.00
3    4        1  2025-08-04  51500.00
4    5        1  2025-08-05  52000.00
5    6        2  2025-08-01   3000.00
6    7        2  2025-08-02   3050.00
7    8        2  2025-08-03   3100.00
8    9        2  2025-08-04   2950.00
9   10        2  2025-08-05   3200.00
10  11        3  2025-08-01      1.50
11  12        3  2025-08-02      1.55
12  13        3  2025-08-03      1.60
13  14        3  2025-08-04      1.45
14  15        3  2025-08-05      1.65


In [6]:
# Step 5: Business question
# Question: What is the average price and highest price for each coin?

query = """
SELECT 
    c.name,
    ROUND(AVG(p.price), 2) AS avg_price,
    MAX(p.price) AS highest_price
FROM prices p
JOIN coins c ON p.coin_id = c.id
GROUP BY c.name
"""

df_result = pd.read_sql_query(query, conn)
print("Business Question Result: Average and Highest Price per Coin")
print(df_result)


Business Question Result: Average and Highest Price per Coin
       name  avg_price  highest_price
0   Bitcoin   50600.00       52000.00
1   Cardano       1.55           1.65
2  Ethereum    3060.00        3200.00


In [7]:
# Step 6: Close connection
conn.close()
print("Database connection closed.")


Database connection closed.
